# NanoPyx "Codeless" Jupyter Notebook

### This notebook allows you to perform channel registration of multi-channel image stacks

### If you found this work useful for your research please consider citing: https://doi.org/10.5281/zenodo.7699280

**Channel Registration** Romain F Laine et al 2019 J. Phys. D: Appl. Phys. 52 163001

## Notebook setup cell

In [ ]:
#@title Install NanoPyx, import necessary libraries and connect to Google Drive
!pip install "nanopyx[all] @git+https://github.com/HenriquesLab/NanoPyx.git" --quiet
import io
import os
import sys
import skimage
import stackview
import numpy as np
import tifffile as tiff
import matplotlib as mpl
import ipywidgets as widgets
from IPython.display import display, clear_output
from matplotlib import pyplot as plt

from nanopyx.methods import channel_registration
from nanopyx.core.utils.easy_gui import EasyGui

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import output
    output.enable_custom_widget_manager()
    from google.colab import drive
    drive.mount('/content/drive')

## Select which image to load

In [ ]:
#@title Create image loader GUI
# Create a GUI
gui_data = EasyGui("Data Loader")

def on_button_load_data_clicked(b):
    clear_output()
    gui_data.show()
    global dataset_original
    # disable button
    gui_data["load_data"].disabled = True
    gui_data["load_data"].description = "Loading..."
    dataset_original = tiff.imread(gui_data["upload"].selected)
    gui_data["load_data"].disabled = False
    gui_data["load_data"].description = "Load data"
    display(stackview.slice(dataset_original, colormap=gui_data["cmaps"].value,
                            continuous_update=True))

gui_data.add_label("Select data to use:")
gui_data.add_file_upload("upload")
gui_data.add_dropdown("cmaps", description="Colormap:",
                      options=sorted(list(mpl.colormaps)),
                      value="viridis", remember_value=True))
gui_data.add_button("load_data", description="Load data")
gui_data["load_data"].on_click(on_button_load_data_clicked)
gui_data.show()

## Now let's perform the registration

In [ ]:
#@title Create channel registration GUI
gui_reg = EasyGui("Channel Registration")

def on_button_register(b):
    clear_output()
    gui_reg.show()
    ref_channel = gui_reg["ref"].value
    max_shift = gui_reg["max"].value
    n_blocks = gui_reg["blocks"].value
    min_sim = gui_reg["min_sim"].value
    global dataset_registered
    gui_reg["register"].disabled = True
    gui_reg["register"].description = "Aligning..."
    dataset_registered = channel_registration.estimate_channel_registration(dataset_original,
                                                                            ref_channel,
                                                                            max_shift,
                                                                            n_blocks,
                                                                            min_sim,
                                                                            save_translation_masks=False,
                                                                            save_ccms=False,
                                                                            apply=True)
    gui_reg["register"].disabled = False
    gui_reg["register"].description = "Align"
    display(stackview.slice(dataset_registered, colormap=gui_reg["cmaps"].value,
                            continuous_update=True))

gui_reg.add_label("Channel Registration parameters:")
gui_reg.add_int_slider("ref", description="Reference channel", min=0, max=dataset_original.shape[0]-1, value=0)
gui_reg.add_int_slider("max", description="Max expected drift", min=0, max=1000, value=10)
gui_reg.add_int_slider("blocks", description="Blocks per axis", min=1, max=10, value=5)
gui_reg.add_float_slider("min_sim", description="Minimum similarity", min=0, max=1, value=0.5, step=0.1)
gui_reg.add_dropdown("cmaps", description="Colormap:",
                     options=sorted(list(mpl.colormaps)),
                     value="viridis", remember_value=True)))
gui_reg.add_button("register", description="Register")
gui_reg["register"].on_click(on_button_register)
gui_reg.show()